# Gradient Boosting Learning

## Introduction

Gradient boosting is one of the most powerful techniques devloped in modern machine learning. It applies for both regression and classification, and despite the similarity with bagging, is a trully different method. It is an ensemble method formed only of weak learners, in general trees, combine in a sequential manner, so that the combination results in a strong learner. 

A weak learner is a predictor that is just slightly better than a random guessing. The idea of boosting is to sequentially apply the weak classification algorithm to a repeteadly modified version of the dataset producing a sequence of weak classifiers $G_m(\vec{x}),\: m=1,2,...,M.$ The modified version arise from the weighted combination of the predictors:

$$
G(\vec{x}) = \sum_{m=1}^M \alpha_m G_m(\vec{x})
$$

in which the weights $\alpha_m$ are defined by the boosting algorithm in such manner that the instances that are harder to predict receives more atention than the ones that are easier. The modification address weights not only to the predictors but to the instances $(x_i,y_i), \: i=1,2,...,N$. All instances are initialized with equal weights $w_i = 1/N$. In the following executions, they are re-weighted so that the harder to predict, the sequential prediction focused in the higher weighted instances, so that the $G_i(\vec{x})$ attempts to correct the errors of the $G_{i-1}(\vec{x})$.

The reason why boosting is a success isn't not difficult to figure out. They key issue is that boosting is a way of fitting an additive expansion in a set of elementary "basis" function where the weak learner does the job of basis. In this sense, we recast the boosting definition as:

$$
f(\vec{x}) = \sum_{m=1}^M \beta_mb(\vec{x},\gamma_m),
$$

where $\beta_m, \: m=1,2,...,M$ are the expansion coefficients, and $b(\vec{x},\gamma_m) \in \mathbb{R}$ are some algebraic simple functions of the multivariate feature vector, characterized by the set of parameters $\gamma$. They are the heart and soul of several learning algorithms:

* Single-hidden-layer neural networks: $b(\vec{x},\gamma) = \sigma(\gamma_0 + \vec{\gamma}_1^T\cdot\vec{x} )$, where $\sigma(\cdot)$ is the sigmoid function.

* Multivariate adaptative regression splines, uses truncated power splines basis function where $\sigma$ parametrizes the variables and values for the knots.

* For trees, the parameters $\gamma$ defines the split variables and the split points at the internal nodes and the predictions of the terminal nodes.

As usual, these models are fitted by minimizing a loss function averaged over the training data using likelihood-based loss function:

$$
min_{\{\beta_m,\gamma_m\}_1^M}\sum_{i=1}^N L\left(y_i,\sum_{m=1}^M\beta_mb(\vec{x}_i,\gamma_m)\right),
$$

For many loss functions, the likelihood and/or basis functions, this requeires computationally intensive numerical optimization techniques. 


## Foward Stagewise, Addtive Modeling, Exponential Loss and AdaBoost

A possible solution for the optimization problem is to sequentialy add new basis functions to the expansion, without adjusting the parameters and coefficients of those that have alredy been added. At each the $m-$th iteration, the algorithm solves for the optimal basis function $b(\vec{x},\gamma_m)$ and corresponding coefficient $\beta_m$ to the current expansion $f_{m-1}(\vec{x})$, producing the $f_m(\vec{x})$. The proces is thus repeated until convergence, and previously added terms are not modified. For the squared-loss function, we have:

$$
\begin{aligned}
L(y_i,f_{m-1}(\vec{x})+\beta b(\vec{x},\gamma)) &= (y_i - f_{m-1}(\vec{x}) - \beta b(\vec{x},\gamma))^2 \\
                                                &= (r_{im} - \beta b(\vec{x},\gamma))^2,
\end{aligned}
$$

thus, at each step, the optimization procedure takes as input the residue of the $m-1$ estimation. The new basis function that best fit the residues is included in the model expansion. This foward stagewise procedure can be applied to any kind of acceptable loss function. Each case corresponds to an algorithm. The seminal boosting method, Adaptative Boosting, AdaBoost for short, can be derived in this way, using an exponential loss function.

Consider the so-called exponential loss function:

$$
L(y,f(\vec{x})) = exp(-y\times f(\vec{x})),
$$
and we assume the basis function to be individual predictors $G_m(\vec{x})$. One must solve:

$$
\begin{aligned}
(\beta_m,G_m) &= arg min_{\beta,G}\sum_{i=1}^N exp\left[-y_i(f_{m-1}(\vec{x})+\beta G(\vec{x}))\right]\\
              &= arg min_{\beta,G}\sum_{i=1}^N w_i^{(m)}exp\left[-y_i(\beta G(\vec{x}))\right],
\end{aligned}
$$
since $w_i^{(m)} = exp\left[-y_i(f_{m-1}(\vec{x}))\right]$ does not depends on neither $\beta$ nor $G(\vec{x})$ it can be factor out as a weight factor applied to each instance. For a classification problem, the $m-th$ $\beta$ is obtained as:

$$
err_{m} = \frac{\sum_{i=1}^Nw_{i}^{(m)}I(y_i \neq G_m(\vec{x}))}{\sum_{i=1}^Nw_i^{(m)}}.
$$

The foward prediction is given by:

$$
f_m(\vec{x}) = f_{m-1}(\vec{x}) + \beta_{m}G_m(\vec{x}),
$$
which results into the following update rule:

$$
w_i^{(m+1)} = w_i^{(m)}exp(2\beta_mI(y_i \neq G_m(\vec{x})) - \beta_m)
$$

## Boosting Trees

Just as a reminder, classification and regression trees, as discussed before, are partitions of the feature space off all joint independent variables into disjoint regions, represented as the terminal nodes in which a prediction is made, based on the average of the region. Thus, a tree model partition the space and assign to each partition a constant value:

$$
x \in R_j \longrightarrow f(x) = \gamma_j
$$

A tree can be formally expressed as:

$$
T(\vec{x},\Theta) = \sum_{j=1}^J\gamma_jI(\vec{x} \ in R_j),
$$
where $\Theta = \{R_j,\gamma_j\}_ 1^J$. In general, $J$ represent all possible combinations of tree hyperparameters and are determined by a combination os cross-validation and empirical risk minimization:

$$
\hat{\Theta} = argmin_{\Theta}\sum_{j=1}^J\sum_{x_i\in R_j}L(y_i, \gamma_j)
$$

This is a complicated optimization problem and in general we relies upon suboptimal solutions. We divide this problem into two parts. Forest of all, we find $\gamma_j$ given $R_j$. This typically trivial and in general we set $\gamma_j = \bar{y}_j$. Than we follow to the second part, finding $R_j$ itself. This is the difficult part. In the tree study, we relied upon greedy solutions under the CART method. Finding $R_j$ requires a first estimation of $\gamma_j$ itself.

Boosted tree models is a sum of these trees:

$$
f_M(\vec{x}) = \sum_{m=1}^M T(\vec{x},\Theta_m),
$$
using the forward stagewise approach. At each step, one must solve:

$$
\hat{\Theta}_m  = arg min_{\Theta_m}\sum_{i=1}^N L(y_im f_{m-1}(\vec{x}) + T(\vec{x},\Theta_m)).
$$

Once the region and the hyperparameters were found, one can simply determine the stage prediction:

$$
\hat{\gamma}_{jm} = argmin_{\gamma_{jm}}\sum_{x_j\in R_j}L(y_i, f_{m-i}(x_i) + \gamma_{jm}).
$$

Now, our next step is moving beyond the greedy solution for these tree problems and unleash the true power of trees and boosting using numerical optimization. The idea is to use a fast approximate algorithm to solve the empirical minimization problem:
$$
L(f) = \sum_{i=1}^NL(y_i,f(x_i)),
$$
with respect to a f function that is a weighted sum of trees, assuming that the parameters are the trees itself:

$$
\hat{f} = arg min_{f}L(f) \longrightarrow d = \{f(\vec{x}_1),...,f(\vec{x}_N)\}.
$$
The solution here has to be more sophisticated than the approach used to loss function and maximum likelihood. We need a numerical optimization method. The most used one is the so-called Gradient Descent.

## Gradient Descent

Gradient descent is a generic numerical optimization method used to find solutions to a wide range of problems by tweaking parameters iteratively towards a minimization of cost function. The fundamental aspect of Gradient Descent is the size of the steps, determined by the learning rate hyperparameter. A too small learning rate, means that the algorithm has to go many interactions to converge whereas if the learning rate is too high, you might jump across the potential landscape without never reaching a solution.

In machine learning, optimization methods plays a fundamental role even performing one single task: minimize the cost function: $E(\theta) = \mathcal{C}(\vec{x},f(\vec{x},\theta))$. The simple Gradient Descent approach, we update the parameters using the following rule:
$$
\vec{v}_t = \eta_t \nabla_{\theta}E(\theta_t),
$$
$$
\vec{\theta}_{t+1} = \vec{\theta}_t - \vec{v}_t,
$$
where $\nabla_{\theta}E(\theta_t)$ is the gradient of the cost function and $\eta_t$ is the learning rate. For a sufficient small choice of the learning rate, the algorithm will converge to a local minimum of the cost function. However, this choice comes with the tradeoff of a high computational cost, since the smaller the $\eta_t$ more steps we have to take to reach a solution. The overshooting choice, large learning rate, brings instabilities to the algorithm and it may never converge. Despite of it conceptual simplicity and practice, Gradient Descent has some limitations that needs to be accounted for:

1. Gradient Descent is a deterministic algorithm. This implies that if ti converges, it will converge for a local minimum. In machine learning we often deals with landscapes with many local minimum, so Gradient Descent can lead to poor performance.
2. Gradient are computationally cost for large datasets
3. Gradient Descent is sensitive to choices of the learning rates.
4. Treats all directions in parameter space uniformly
5. Sensitive to initial conditions
6. Take exponential time to scape saddles

Many of these shortcomings can be addressed by a few variants of Gradient Descent. The most famous one is the so-called Stochastic Gradient Descent. One cand introduce stochasticity into gradient descent by approximating the gradient in a subset of data called minibatch. This subset is always much smaller than the total number of datapoints. Thus, in this approach, each gradient descent step, we approximate the gradient using a minibatch $B_k$:

$$
\nabla_{\theta}E(\theta) = \sum_{i=1}^n\nabla_{\theta}e_i(\vec{x}_i,\theta) \longrightarrow \sum_{i \in B_k}^n\nabla_{\theta}e_i(\vec{x}_i,\theta).
$$

We then cycle over all $k= 1, ..., n/M$ minibatches one at time, and use the mini-batch approximation to the gradient ot update the parameters $\theta$ at every step $k$. We call a full iteration an epoch. We write the SGD algorithm over the minibatche procedure as:

$$
\vec{v}_t = \eta_t\nabla_{\theta}E^{MB}(\theta),
$$
$$
\theta_{t+1} = \theta_t - \vec{v}_t.
$$

The Stochastic Gradient Algorithm we replace the gradient over the full data at each step by an approximation calculated using the minibatch. The stochasticity is introduced by the sampling over tha mini batches, reducing the chance of the algorithm gets stuck in isolated local minima and speeds up the calculation as one does not need to use all datapoints. Also, the stochasticity reduces the chance of overfitting.

In [24]:

# Import pandas and numpy
import pandas as pd
import numpy as np

# Silence warnings
import warnings
warnings.filterwarnings('ignore')

# Importing the model evaluation commands
from sklearn.model_selection import(
    train_test_split,
    cross_val_score,
    cross_val_predict,
    GridSearchCV,
    RandomizedSearchCV
)
from sklearn.metrics import (
    mean_squared_error,
    r2_score,
    explained_variance_score,
    roc_curve,
    auc
)

In [21]:
# Define Functions That Evaluate Model Performance
##########################- Root Mean Square -####################################
def rmse(model, x_val, y_val):
    pred = model.predict(x_val)
    return np.sqrt(mean_squared_error(y_val, pred))
##########################- R2 Score -###########################################
def r2(model, x_val, y_val):
    pred = model.predict(x_val)
    return r2_score(y_val, pred)
##########################- Accuracy -###########################################
def accuracy(model, x_val, y_val):
    pred = model.predict(x_val)
    return r2_score(y_val, pred)
##########################- Precision -##########################################
def precision(model, x_val, y_val):
    pred = model.predict(x_val)
    return r2_score(y_val, pred)
##########################- Recall -#############################################
def recall(model, x_val, y_val):
    pred = model.predict(x_val)
    return r2_score(y_val, pred)
##########################- Classification Repor -###############################
def report(model, x_val, y_val):
    pred = model.predict(x_val)
    return r2_score(y_val, pred)
##########################- ROC -################################################
def roc(model, x_val, y_val):
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(y_test))]
    # predict probabilities
    lr_probs = dt.predict_proba(X_test)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_test, ns_probs)
    lr_auc = roc_auc_score(y_test, lr_probs)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Model: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label='Model')
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    return plt.show()
##########################- Validation Plot -####################################
def valid_reg(model, x_val, y_val):
    # Place an orange "X" on the point with the best Sharpe ratio
    pred = model.predict(x_val)
    fig, ax = plt.subplots()
    plt.scatter(x=y_val, y=pred,alpha=0.5,color='green')
    line = mlines.Line2D([0, 1], [0, 1], color='black')
    transform = ax.transAxes
    line.set_transform(transform)
    ax.add_line(line)
    plt.grid()
    plt.xlabel('Target')
    plt.ylabel('Prediction')
    return plt.show()

In [2]:
df_bikes = pd.read_csv('bike_rentals_cleaned.csv')
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [3]:
# Split data into X and y
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

# Import train_test_split
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [4]:
# Import Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor

# Initialize Decision Tree Regressor
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)

# Fit tree to training data
tree_1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [5]:
# Make predictions on training set
y_train_pred = tree_1.predict(X_train)

In [6]:
# Compute residuals
y2_train = y_train - y_train_pred

# Initialize Decision Tree Regressor
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)

# Fit tree to training data
tree_2.fit(X_train, y2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [7]:
# Make predictions on training set
y2_train_pred = tree_2.predict(X_train)

# Compute residuals
y3_train = y2_train - y2_train_pred

# Initialize Decision Tree Regressor
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)

# Fit tree to training data
tree_3.fit(X_train, y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [8]:
y1_pred = tree_1.predict(X_test)

y2_pred = tree_2.predict(X_test)

y3_pred = tree_3.predict(X_test)

y_pred = y1_pred + y2_pred + y3_pred

# Import mean_squared_error
from sklearn.metrics import mean_squared_error as MSE

# Compute root mean squared error (rmse)
MSE(y_test, y_pred)**0.5


911.0479538776444

In [1]:
from sklearn.ensemble import GradientBoostingRegressor

In [26]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=2, learning_rate=1.0)

gbr.fit(X_train, y_train)

# Predict test data
y_pred = gbr.predict(X_test)

# Compute root mean squared error (rmse)
#MSE(y_test, y_pred)**0.5
print(f"rmse on training set: {rmse(gbr, X_train, y_train)}")
print(f"R2 on training set: {r2(gbr, X_train, y_train)}")
print(f"rmse on test set: {rmse(gbr,X_test,y_test)}")
print(f"R2 on test set: {r2(gbr,X_test,y_test)}")

rmse on training set: 801.1808431544883
R2 on training set: 0.8259707682214588
rmse on test set: 911.0479538776439
R2 on test set: 0.7873491063875538


In [27]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
# Compute root mean squared error (rmse)
#MSE(y_test, y_pred)**0.5
print(f"rmse on training set: {rmse(gbr, X_train, y_train)}")
print(f"R2 on training set: {r2(gbr, X_train, y_train)}")
print(f"rmse on test set: {rmse(gbr,X_test,y_test)}")
print(f"R2 on test set: {r2(gbr,X_test,y_test)}")

rmse on training set: 437.00944371671716
R2 on training set: 0.9482223024358075
rmse on test set: 857.1072323426944
R2 on test set: 0.8117846423175021


In [28]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
print(f"rmse on training set: {rmse(gbr, X_train, y_train)}")
print(f"R2 on training set: {r2(gbr, X_train, y_train)}")
print(f"rmse on test set: {rmse(gbr,X_test,y_test)}")
print(f"R2 on test set: {r2(gbr,X_test,y_test)}")

rmse on training set: 38.12518194485307
R2 on training set: 0.9996059195049195
rmse on test set: 936.3617413678853
R2 on test set: 0.7753677748365476


In [29]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
print(f"rmse on training set: {rmse(gbr, X_train, y_train)}")
print(f"R2 on training set: {r2(gbr, X_train, y_train)}")
print(f"rmse on test set: {rmse(gbr,X_test,y_test)}")
print(f"R2 on test set: {r2(gbr,X_test,y_test)}")

rmse on training set: 328.6357792505077
R2 on training set: 0.9707186713820268
rmse on test set: 653.7456840231495
R2 on test set: 0.890502952905042


In [14]:
learning_rate_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]
for value in learning_rate_values:
    gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=value)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Learning Rate:', value, ', Score:', rmse)

Learning Rate: 0.001 , Score: 1633.0261400367258
Learning Rate: 0.01 , Score: 831.5430182728547
Learning Rate: 0.05 , Score: 685.0192988749717
Learning Rate: 0.1 , Score: 653.7456840231495
Learning Rate: 0.15 , Score: 687.666134269379
Learning Rate: 0.2 , Score: 664.312804425697
Learning Rate: 0.3 , Score: 689.4190385930236
Learning Rate: 0.5 , Score: 693.8856905068778
Learning Rate: 1.0 , Score: 936.3617413678853


In [15]:
depths = [None, 1, 2, 3, 4]
for depth in depths:
    gbr = GradientBoostingRegressor(max_depth=depth, n_estimators=300, random_state=2)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Max Depth:', depth, ', Score:', rmse)

Max Depth: None , Score: 869.2788645118395
Max Depth: 1 , Score: 707.8261886858736
Max Depth: 2 , Score: 653.7456840231495
Max Depth: 3 , Score: 646.4045923317708
Max Depth: 4 , Score: 663.048387855927


In [16]:
samples = [1, 0.9, 0.8, 0.7, 0.6, 0.5]
for sample in samples:
    gbr = GradientBoostingRegressor(max_depth=3, n_estimators=300, subsample=sample, random_state=2)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Subsample:', sample, ', Score:', rmse)

Subsample: 1 , Score: 646.4045923317708
Subsample: 0.9 , Score: 620.1819001443569
Subsample: 0.8 , Score: 617.2355650565677
Subsample: 0.7 , Score: 612.9879156983139
Subsample: 0.6 , Score: 622.6385116402317
Subsample: 0.5 , Score: 626.9974073227554


In [30]:
params={'subsample':[0.65, 0.7, 0.75],
        'n_estimators':[300, 500, 1000],
        'learning_rate':[0.05, 0.075, 0.1]
        }

# Import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

gbr = GradientBoostingRegressor(max_depth=3, random_state=2)


# Instantiate RandomizedSearchCV as rand_reg
rand_reg = RandomizedSearchCV(gbr, params, n_iter=10, scoring='neg_mean_squared_error',
                              cv=5, n_jobs=-1, random_state=2)

# Fit grid_reg on X_train and y_train
rand_reg.fit(X_train, y_train)

# Extract best estimator
best_model = rand_reg.best_estimator_

# Extract best params
best_params = rand_reg.best_params_

# Print best params
print("Best params:", best_params)

# Compute best score
best_score = np.sqrt(-rand_reg.best_score_)

# Print best score
print("Training score: {:.3f}".format(best_score))

# Predict test set labels
y_pred = best_model.predict(X_test)

print(f"rmse on training set: {rmse(best_model, X_train, y_train)}")
print(f"R2 on training set: {r2(best_model, X_train, y_train)}")
print(f"rmse on test set: {rmse(best_model,X_test,y_test)}")
print(f"R2 on test set: {r2(best_model,X_test,y_test)}")

Best params: {'subsample': 0.65, 'n_estimators': 300, 'learning_rate': 0.05}
Training score: 636.200
rmse on training set: 269.10852165028706
R2 on training set: 0.9803656742918951
rmse on test set: 625.9849010532475
R2 on test set: 0.8996049144921957


In [31]:
gbr = GradientBoostingRegressor(max_depth=3, n_estimators=1600, subsample=0.75, learning_rate=0.02, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
print(f"rmse on training set: {rmse(gbr, X_train, y_train)}")
print(f"R2 on training set: {r2(gbr, X_train, y_train)}")
print(f"rmse on test set: {rmse(gbr,X_test,y_test)}")
print(f"R2 on test set: {r2(gbr,X_test,y_test)}")

rmse on training set: 159.90255545058218
R2 on training set: 0.9930677869910395
rmse on test set: 596.9544588974487
R2 on test set: 0.9087007649429504


In [32]:
# Import XGBRegressor
from xgboost import XGBRegressor

# Instantiate the XGBRegressor, xg_reg
xg_reg = XGBRegressor(max_depth=3, n_estimators=1600, eta=0.02, subsample=0.75, random_state=2)

# Fit xg_reg to training set
xg_reg.fit(X_train, y_train)

# Predict labels of test set, y_pred
y_pred = xg_reg.predict(X_test)

# Compute root mean squared error (rmse)
print(f"rmse on training set: {rmse(xg_reg, X_train, y_train)}")
print(f"R2 on training set: {r2(xg_reg, X_train, y_train)}")
print(f"rmse on test set: {rmse(xg_reg,X_test,y_test)}")
print(f"R2 on test set: {r2(xg_reg,X_test,y_test)}")

rmse on training set: 180.53055516221195
R2 on training set: 0.99116386127466
rmse on test set: 584.3395337495713
R2 on test set: 0.9125186901052338
